In [14]:
import json
import time
import azureml
import logging
from azureml.core.model import Model
from azureml.train.automl import AutoMLConfig
from azureml.core import Workspace, Run, Datastore, Experiment
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.train.hyperdrive import *
from azureml.train.dnn import TensorFlow
from azureml.widgets import RunDetails

print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  0.1.80


In [15]:
# use this code to set up config file
#subscription_id ='<SUB_ID>'
#resource_group ='<RESOURCE_GROUP>'
#workspace_name = '<WORKSPACE>'

#try:
#   ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
#   ws.write_config()
#   print('Workspace configuration succeeded. You are all set!')
#except:
#   print('Workspace not found. TOO MANY ISSUES!!!')
ws = Workspace.from_config()

Found the config file in: C:\projects\SimpleMNIST\aml_config\config.json


# Compute Environment

In [16]:
cluster = 'sauron'
try:
    compute = ComputeTarget(workspace=ws, name=cluster)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', min_nodes=1, max_nodes=6)
    compute = ComputeTarget.create(ws, cluster, compute_config)
    compute.wait_for_completion(show_output=True)
    

Found existing compute target


# Run Experiment

In [ ]:
# Create and run experiment
mnist = Experiment(ws, 'simplemnist')
estimator = TensorFlow(source_directory='.',
                       compute_target=compute,
                       entry_script='train.py',
                       use_gpu=True)

run = mnist.submit(estimator)

In [ ]:
run

In [ ]:
RunDetails(run).show()

# Manage Data

In [4]:
ds = ws.get_default_datastore()
mnist_data = ds.upload(src_dir = 'data', target_path = 'mnist', show_progress = True)

Target already exists. Skipping upload for mnist\mnist.npz


In [ ]:
# run the same was as above
script_params={
    '--data': mnist_data.as_mount(),
}

# Create and run experiment
mnist = Experiment(ws, 'simplemnist')
estimator = TensorFlow(source_directory='.',
                       compute_target=compute,
                       entry_script='train.py',
                       script_params=script_params,
                       use_gpu=True)

run = mnist.submit(estimator)
RunDetails(run).show()

# Optimize hyperparameters

In [5]:
# same as above but increase the max_steps and remove the parameters
script_params={
    '--data': mnist_data,
    '--epochs': 100
}

mnist = Experiment(ws, 'simplemnist')
estimator = TensorFlow(source_directory='.',
                       compute_target=compute,
                       entry_script='train.py',
                       script_params=script_params,
                       use_gpu=True)

## hyperparameter search

In [6]:
ps = RandomParameterSampling(
    {
        '--lr': loguniform(-15, -3),
        '--batch': choice(16, 32, 64, 128)
    }
)

early_termination_policy = BanditPolicy(slack_factor = 0.15, evaluation_interval=2)

hyperdrive_run_config = HyperDriveRunConfig(estimator = estimator, 
                                            hyperparameter_sampling = ps, 
                                            policy = early_termination_policy,
                                            primary_metric_name = "accuracy",
                                            primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                                            max_total_runs = 20,
                                            max_concurrent_runs = 5)

hd_run = mnist.submit(hyperdrive_run_config)

RunDetails(hd_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'NOTSE…

In [ ]:
#hd_run.cancel()

# Automatic Machine Learning

In [21]:
automl_settings = {
    "name": "AutoML_Demo_Experiment_{0}".format(time.time()),
    "iteration_timeout_minutes": 5,
    "iterations": 20,
    "n_cross_validations": 5,
    "primary_metric": 'AUC_weighted',
    "preprocess": False,
    "max_concurrent_iterations": 10,
    "verbosity": logging.INFO
}

automl_config = AutoMLConfig(task='classification',
                             debug_log='automl_errors.log',
                             path='.',
                             compute_target = compute,
                             data_script='./get_data.py',
                             **automl_settings)

In [22]:
experiment=Experiment(ws, 'simplemnist')
remote_run = experiment.submit(automl_config, show_output=True)

Running on remote compute: sauron
Parent Run ID: AutoML_b3539306-d3d5-45e8-b131-78f568245759
*******************************************************************************************
ITERATION: The iteration being evaluated.
PIPELINE: A summary description of the pipeline being evaluated.
DURATION: Time taken for the current iteration.
METRIC: The result of computing score on the fitted pipeline.
BEST: The best observed score thus far.
*******************************************************************************************

 ITERATION   PIPELINE                                       DURATION      METRIC      BEST
         0   MaxAbsScaler LightGBM                          0:05:02       0.9935    0.9935
         1   StandardScalerWrapper LightGBM                 0:09:18       0.9964    0.9964
         2   StandardScalerWrapper LogisticRegression       0:12:36       0.9877    0.9964
         4   RobustScaler LogisticRegression                0:14:09       0.9893    0.9964
         3

In [24]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'NOTSET', …